# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

In [1]:
import nltk

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [2]:
text = open('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt').read()
sents = nltk.sent_tokenize(text)
sents[:3]

['Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения!',
 'Федор Михайлович Достоевский\n\nИгрок\n\n(Из записок молодого человека)\n\n\n\n\nГлава I\n\nНаконец я возвратился из моей двухнедельной отлучки.',
 'Наши уже три дня как были в Рулетенбурге.']

In [3]:
[len(sent) for sent  in sents][:3]

[287, 133, 41]

In [4]:
list(map(len, sents))[:3]

[287, 133, 41]

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [5]:
[sent for sent  in sents if len(sent)<=10][:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

In [7]:
list(filter(lambda sent: len(sent)<=10, sents))[:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [10]:
import dask.bag as db

In [15]:
bag = db.from_sequence(sents)
bag.map(len).mean().compute()

72.89840944073885

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [16]:
import json

In [17]:
data = json.load(open('addres-book.json'))
data[0]

{'name': 'Faina Lee',
 'email': 'faina@mail.ru',
 'birthday': '22.08.1994',
 'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]}

In [18]:
bag = db.from_sequence(data)
bag

dask.bag<from_sequence, npartitions=8>

In [22]:
bag.map(lambda record: record['phones']).compute()

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}],
 [{'phone': '232-19-56'}, {'phone': '+7 (916) 232-19-56'}],
 [{'phone': '+7 (916) 445-19-57'}],
 [{'phone': '232-19-58'}, {'phone': '+7 (916) 232-19-58'}],
 [{'phone': '+7 (916) 445-19-59'}],
 [{'phone': '232-19-50'}, {'phone': '+7 (916) 232-19-50'}],
 [{'phone': '111-19-51'}, {'phone': '+7 (916) 445-19-51'}]]

In [25]:
bag.pluck('phones').flatten().compute()

[{'phone': '232-19-55'},
 {'phone': '+7 (916) 232-19-55'},
 {'phone': '111-19-55'},
 {'phone': '232-19-56'},
 {'phone': '+7 (916) 232-19-56'},
 {'phone': '+7 (916) 445-19-57'},
 {'phone': '232-19-58'},
 {'phone': '+7 (916) 232-19-58'},
 {'phone': '+7 (916) 445-19-59'},
 {'phone': '232-19-50'},
 {'phone': '+7 (916) 232-19-50'},
 {'phone': '111-19-51'},
 {'phone': '+7 (916) 445-19-51'}]

In [36]:
def add_type(record):
	record['type'] = 'mobile' if record['phone'].startswith('+7') else 'work'
	return record

In [37]:
bag.pluck('phones').flatten().map(add_type).compute()

[{'phone': '232-19-55', 'type': 'work'},
 {'phone': '+7 (916) 232-19-55', 'type': 'mobile'},
 {'phone': '111-19-55', 'type': 'work'},
 {'phone': '232-19-56', 'type': 'work'},
 {'phone': '+7 (916) 232-19-56', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-57', 'type': 'mobile'},
 {'phone': '232-19-58', 'type': 'work'},
 {'phone': '+7 (916) 232-19-58', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-59', 'type': 'mobile'},
 {'phone': '232-19-50', 'type': 'work'},
 {'phone': '+7 (916) 232-19-50', 'type': 'mobile'},
 {'phone': '111-19-51', 'type': 'work'},
 {'phone': '+7 (916) 445-19-51', 'type': 'mobile'}]

In [31]:
bag.pluck('phones').flatten().map(lambda record: 'mobile' if record['phone'].startswith('+7') else 'work').frequencies().compute()

[('work', 6), ('mobile', 7)]

In [39]:
(bag.pluck('phones')
    .flatten()
    .map(add_type)
    .pluck('type')
    .frequencies()
    .compute())

[('work', 6), ('mobile', 7)]

In [40]:
phones_with_types = bag.pluck('phones').flatten().map(add_type)
type_freqs = phones_with_types.pluck('type').frequencies()
type_freqs.compute()

[('work', 6), ('mobile', 7)]

In [33]:
def binop(total, value):
	return total + 1

def combine(left, right):
	return left + right

In [34]:
bag.pluck('phones')\
    .flatten()\
    .map(add_type)\
	.foldby('type', binop, 0, combine, 0)\
	.compute()

TypeError: 'NoneType' object is not subscriptable

Traceback
---------
  File "D:\Programming\Python\BigDataPT-BDPT-\venv\lib\site-packages\dask\local.py", line 220, in execute_task
    result = _execute_task(task, data)
  File "D:\Programming\Python\BigDataPT-BDPT-\venv\lib\site-packages\dask\core.py", line 119, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
  File "D:\Programming\Python\BigDataPT-BDPT-\venv\lib\site-packages\toolz\itertoolz.py", line 623, in reduceby
    k = key(item)


## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

3. Посчитайте количество отзывов в исходном датасете.

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.